In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import bnlearn as bn

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
from utils import read_data
combined_data = read_data()

In [ ]:
select_group = [

    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    # '車輛撞擊部位大類別名稱-最初',
    # '事故類型及型態大類別名稱', '車道劃分設施-分向設施大類別名稱',
    # '道路型態大類別名稱',
    '車輛撞擊部位子類別名稱-最初',
    '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    '道路型態子類別名稱',

    # 其他
    '速限-第1當事者', '道路類別-第1當事者-名稱',

    # 設施
    'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',
    
    # 駕駛、行人行為
    '肇因研判子類別名稱-主要'
    ]

In [ ]:
data = combined_data[select_group].copy()
data['facility'] = data[['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count']].apply(
    lambda row: '1' if (row > 0).any() else '0', axis=1
)
data.drop(columns=['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count'], inplace=True)
data['速限-第1當事者'] = data['速限-第1當事者'].apply(lambda x: 'High' if x > 50 else 'Low')
# 專注分析市區道路
data = data[data['道路類別-第1當事者-名稱'] == '市區道路']

In [ ]:
for i in data.columns:
    print(i, len(data[i].unique()))

In [ ]:
# 學哪些變數之間有邊，結果是一個DAG
model = bn.structure_learning.fit(data, methodtype='hc', scoretype='bic')
# 計算每個節點的 條件機率表 (CPT, Conditional Probability Table)
model_param = bn.parameter_learning.fit(model, data)
# 計算邊緣強度，如果p小於顯著就是有相關
model_independence = bn.independence_test(model, data, test='chi_square', prune=True)

In [ ]:
model_independence['independence_test']

In [ ]:
bn.get_parents(model['model_edges'])

In [ ]:
# Conditional Probability Distributions (CPDs)
CPDs = bn.print_CPD(model_param)
CPDs['車道劃分設施-分道設施-路面邊線名稱']

In [ ]:
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei']
rcParams['axes.unicode_minus'] = False

# G = bn.plot(model, interactive=False, node_color="#36AA5B", edge_labels=None)
bn.plot(model_independence, interactive=False, edge_labels='pvalue', params_static={'layout': 'spring_layout'})

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

def draw_bn_plotly(model, alpha=0.05, layout_algo=""):
    edges = [(str(u), str(v)) for u, v in model['model_edges']]
    df = model['independence_test'][['source','target','p_value']].copy()
    df['source'] = df['source'].astype(str); df['target'] = df['target'].astype(str)
    p_map = {(s,t):p for s,t,p in df.itertuples(index=False, name=None)}
    p_map.update({(t,s):p for (s,t),p in list(p_map.items())})

    G = nx.DiGraph()
    G.add_edges_from(edges)

    pos = (nx.spring_layout(G, seed=42) if layout_algo=="spring"
           else nx.kamada_kawai_layout(G))

    # nodes
    deg = dict(G.degree()); mdeg = max(deg.values()) if deg else 1
    node_x, node_y, node_text, node_size = [], [], [], []
    for n in G.nodes():
        x,y = pos[n]
        node_x.append(x); node_y.append(y)
        node_text.append(f"{n}<br>degree: {deg.get(n,0)}")
        node_size.append(10 + 25*(deg.get(n,1)/mdeg))

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=[str(n) for n in G.nodes()], textposition="top center",
        hovertext=node_text, hoverinfo="text",
        marker=dict(size=node_size, line=dict(width=1), color="#636efa")
    )

    # edges
    edge_traces = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        p = p_map.get((u,v), np.nan)
        if np.isnan(p):
            width, dash, color = 1.0, "dot", "#999"
            tip = f"{u} → {v}<br>p-value: N/A"
        else:
            w = -np.log10(max(p, 1e-300))
            width = 1 # 1 + 0.8*min(10, w)
            sig = (p <= alpha)
            dash = "solid" if sig else "dot"
            color = "#d62728" if sig else "#1f77b4"
            tip = f"{u} → {v}<br>p-value: {p:.3e}"

        edge_traces.append(go.Scatter(
            x=[x0, x1], y=[y0, y1],
            mode='lines',
            hoverinfo='text', text=[tip],
            line=dict(width=width, color=color, dash=dash)
        ))

        # 箭頭（用 annotation 畫，避免太多圖層負擔）
    annotations = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        annotations.append(dict(
            ax=x0, ay=y0, x=x1, y=y1,
            xref="x", yref="y", axref="x", ayref="y",
            showarrow=True, arrowhead=3, arrowsize=1.2, opacity=0.8
        ))

    fig = go.Figure(data=edge_traces + [node_trace],
        layout=go.Layout(
            template=None, showlegend=False,
            hovermode='closest',
            margin=dict(l=10, r=10, t=10, b=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            annotations=annotations
        )
    )
    return fig

draw_bn_plotly(model_independence, alpha=0.01, layout_algo='w')